In [ ]:
import pandas as pd
import numpy as np
import os
import typing
import tifffile
from tqdm import tqdm

# import the data (here's an old iteration, but it's the same)
pixels = pd.read_hdf("20241007_pixels_featsel_allbrains.h5ad", key="table")
peaks = np.array(pixels.iloc[:,:137].columns.astype(float))
pixels

In [ ]:
# select three lipids that together expose most anatomy to help manual and automated warping to the Allen cell density and contours
targets = np.array([845.528425, 731.606009, 784.657705])

indices = np.abs(peaks[:, np.newaxis] - targets).argmin(axis=0)
closest_values = peaks[indices]
closest_values

In [ ]:
# a function to save in a format compatible with ABBA for CCF registration

def save_imagej_tiff(save_path: str, data: np.ndarray, scale: tuple[float, ...], units: str, com):
    """Save image as tiff to path or buffer
    :param scale: image scale
    :param data: image for save
    :param units: units of image
    :param save_path: save location
    """
    assert data.dtype in [np.uint8, np.uint16, np.float32]
    metadata: dict[str, typing.Any] = {"mode": "color", "unit": units}
    if len(scale) >= 3:
        metadata["spacing"] = scale[-3]
    resolution = [1 / x for x in scale[-2:]]
    tifffile.imwrite(
        save_path,
        data,
        imagej=True,
        metadata=metadata,
        resolution=resolution,
    )  # , compress=6,

In [ ]:
# extract the data

compdata = pixels.loc[:,closest_values.astype(str)] 
compdata = pd.concat([compdata, pixels[['SectionID',	'x',	'y', 'Sample', 'Section', 'Sex']]], axis=1)
compdata.columns = compdata.columns.astype(str)
compdata['x'] = compdata['x'].astype(int)
compdata['y'] = compdata['y'].astype(int)
closest_values = closest_values.astype(str)

In [ ]:
# cure the contrast and export the images in a dynamic range and format to ease the experimemnter's life to do registration to CCF

for sec in tqdm(range(33, 139)):
    now = compdata.loc[compdata['SectionID'] == sec,:]
    
    img = np.full((now['x'].max()+1, now['y'].max()+1), np.nan)

    for _, row in now.iterrows():
        y = row['y']
        z = row['x']
        intensity = row[closest_values[0]]
        img[z, y] = intensity

    p3 = np.nanpercentile(img, 1.5)
    p97 = np.nanpercentile(img, 98.5)
    img = (img - p3) / (p97 - p3)
    img = np.clip(img, 0, 1)
    img = (img * 255).astype(np.uint8)
    img0 = img.copy()

    img = np.full((now['x'].max()+1, now['y'].max()+1), np.nan)

    for _, row in now.iterrows():
        y = row['y']
        z = row['x']
        intensity = row[closest_values[1]]
        img[z, y] = intensity

    p3 = np.nanpercentile(img, 3)
    p97 = np.nanpercentile(img, 97)
    img = (img - p3) / (p97 - p3)
    img = np.clip(img, 0, 1)
    img = (img * 255).astype(np.uint8)
    img1 = img.copy()

    img = np.full((now['x'].max()+1, now['y'].max()+1), np.nan)

    for _, row in now.iterrows():
        y = row['y']
        z = row['x']
        intensity = row[closest_values[2]]
        img[z, y] = intensity

    p3 = np.nanpercentile(img, 3)
    p97 = np.nanpercentile(img, 97)
    img = (img - p3) / (p97 - p3)
    img = np.clip(img, 0, 1)
    img = (img * 255).astype(np.uint8)
    img2 = img.copy()

    composite = np.stack((img0, img1, img2))
    composite_transposed = np.transpose(composite, (1, 2, 0))
    scale = (25, 25, 25)
    units = "um"
    save_imagej_tiff(os.getcwd()+"/ABBA-in/composite_transposed_"+str(now['Sample'].unique())+"_"+str(now['Section'].unique())+"_"+str(now['SectionID'].unique())+".tif", composite_transposed, scale, units, None)

In [ ]:
# zip it to transfer via email
!zip -r ABBA-in.zip ABBA-in